In [ ]:
import importlib
import classify
importlib.reload(classify)
from classify import classify
import torch
from transformers import pipeline
import pandas as pd

In [ ]:
path = './data/raw'
df = pd.DataFrame()

for i in range(8):
    new_df = pd.read_csv(path+f'/Education_videos_{i}_cleaned.csv')
    new_df = new_df.dropna(axis=0)
    df = pd.concat([df, new_df])
    del(new_df)
    print(f'Imported Education_videos_{i}_cleaned.csv')

Imported Education_videos_0_cleaned.csv
Imported Education_videos_1_cleaned.csv
Imported Education_videos_2_cleaned.csv
Imported Education_videos_3_cleaned.csv
Imported Education_videos_4_cleaned.csv
Imported Education_videos_5_cleaned.csv
Imported Education_videos_6_cleaned.csv
Imported Education_videos_7_cleaned.csv


In [23]:
df['upload_date'] = pd.to_datetime(df['upload_date'])
df['year'] = df['upload_date'].dt.year

In [ ]:
N = 50000
data = df.sample(N)

purpose_labels = [
    "academic",
    "edutainment or science popularization",
    "hobby"
]

# Set the device to cuda
device = torch.device('cuda')

# Initialize the BART-based classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# Pre-process text
data['text'] = data['title'] + data['tags']


# Perform classification
print('Start classification...')
final_data = classify(data, candidate_labels=purpose_labels, on='text', classifier=classifier, batch_size=16, multi_label=False)

# Save results
final_data.to_csv(f'data/raw/Education_videos_classified_BART_{N}_3_labels.csv', index=False)
print('CSV saved')

Start classification...
Converting to dataset...
Processing...


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Converting back to DataFrame...
CSV saved
